In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import requests

# table xpath = /html/body/div[2]/div/div/div[1]/div[4]/table/tbody
driver = webdriver.Firefox()
driver.get("https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares.html")
html = driver.page_source
soup = BeautifulSoup(html)
print(soup.prettify())